# Week 8 Assignment - Q-Learning Reinforcement Learning
## Mountain Car Problem - Complete Solution

**Student:** [Your Name Here]
**Date:** October 17, 2025

---

## Overview
This notebook implements Q-Learning for the Mountain Car environment from OpenAI Gymnasium. The car must learn to reach the flag on top of the hill by building momentum through swinging back and forth.

**Key Concepts:**
- **Q-Learning**: Model-free reinforcement learning algorithm
- **Q-Table**: Lookup table storing Q-values for state-action pairs
- **Exploration vs Exploitation**: Balance between trying new actions and using known good actions
- **Reward Function**: Agent receives -1 for each time step until goal is reached

**Environment Details:**
- **State Space**: Position and velocity (continuous)
- **Action Space**: 3 discrete actions (push left, do nothing, push right)
- **Goal**: Reach the flag at position ≥ 0.5

## Part 1: Environment Setup and Exploration

### 1.1 Import Required Libraries
We need gymnasium for the environment, numpy for numerical operations, and matplotlib for visualization.

In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

print("All libraries imported successfully!")
print(f"Gymnasium version: {gym.__version__}")

All libraries imported successfully!
Gymnasium version: 1.2.1


### 1.2 Create and Explore the Environment

The Mountain Car environment presents a challenging problem where the car doesn't have enough power to drive straight up the hill. Instead, it must learn to build momentum by swinging back and forth.

In [2]:
env = gym.make('MountainCar-v0', render_mode=None)

print("Mountain Car Environment Information:")
print("="*60)
print(f"Observation Space: {env.observation_space}")
print(f"  - High: {env.observation_space.high}")
print(f"  - Low: {env.observation_space.low}")
print(f"\nAction Space: {env.action_space}")
print(f"  - Number of actions: {env.action_space.n}")
print(f"  - Actions: 0=Push Left, 1=No Push, 2=Push Right")

state, info = env.reset()
print(f"\nInitial State: {state}")
print(f"  - Position: {state[0]:.4f}")
print(f"  - Velocity: {state[1]:.4f}")

Mountain Car Environment Information:
Observation Space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
  - High: [0.6  0.07]
  - Low: [-1.2  -0.07]

Action Space: Discrete(3)
  - Number of actions: 3
  - Actions: 0=Push Left, 1=No Push, 2=Push Right

Initial State: [-0.41608578  0.        ]
  - Position: -0.4161
  - Velocity: 0.0000


### 1.3 Understanding the Problem

**Why Q-Learning?**
- The car cannot simply drive straight up the hill
- It needs to learn a sequence of actions to build momentum
- Q-Learning helps discover this optimal strategy through trial and error

**Continuous to Discrete Conversion:**
Since Q-Learning uses a table, we need to discretize the continuous state space into manageable buckets.

In [3]:
DISCRETE_OS_SIZE = [40, 40] 

discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_OS_SIZE

print(" State Space Discretization:")
print("="*60)
print(f"Discrete OS Size: {DISCRETE_OS_SIZE}")
print(f"Bucket Size for Position: {discrete_os_win_size[0]:.4f}")
print(f"Bucket Size for Velocity: {discrete_os_win_size[1]:.4f}")
print(f"\nTotal Q-Table Size: {DISCRETE_OS_SIZE[0]} × {DISCRETE_OS_SIZE[1]} × {env.action_space.n}")
print(f"Total Q-values to learn: {DISCRETE_OS_SIZE[0] * DISCRETE_OS_SIZE[1] * env.action_space.n}")

 State Space Discretization:
Discrete OS Size: [40, 40]
Bucket Size for Position: 0.0450
Bucket Size for Velocity: 0.0035

Total Q-Table Size: 40 × 40 × 3
Total Q-values to learn: 4800


## Part 2: Q-Table Initialization

### 2.1 Create the Q-Table

The Q-table stores the expected future rewards for taking each action in each state. Initially, we populate it with random values between -2 and 0, since all rewards in this environment are negative until the goal is reached.

In [4]:
q_table = np.random.uniform(
    low=-2,
    high=0,
    size=(DISCRETE_OS_SIZE + [env.action_space.n])
)

print(" Q-Table Initialized:")
print("="*60)
print(f"Shape: {q_table.shape}")
print(f"Size: {q_table.size} Q-values")
print(f"\nSample Q-values at state (10, 10):")
print(f"  Action 0 (Push Left): {q_table[10][10][0]:.4f}")
print(f"  Action 1 (No Push): {q_table[10][10][1]:.4f}")
print(f"  Action 2 (Push Right): {q_table[10][10][2]:.4f}")

 Q-Table Initialized:
Shape: (40, 40, 3)
Size: 4800 Q-values

Sample Q-values at state (10, 10):
  Action 0 (Push Left): -1.8188
  Action 1 (No Push): -1.9926
  Action 2 (Push Right): -0.6240


### 2.2 Helper Function: Convert Continuous State to Discrete

This function converts the continuous state values (position and velocity) into discrete bucket indices that we can use to look up Q-values in our table.

In [5]:
def get_discrete_state(state):
    """
    Convert continuous state to discrete state (bucket indices).

    Parameters:
    -----------
    state : array
        Continuous state [position, velocity]

    Returns:
    --------
    tuple : Discrete state indices
    """
    discrete_state = (state - env.observation_space.low) / discrete_os_win_size
    return tuple(discrete_state.astype(int))

test_state = np.array([-0.5, 0.0])
discrete_state = get_discrete_state(test_state)
print(f"\n State Conversion Example:")
print(f"Continuous State: {test_state}")
print(f"Discrete State: {discrete_state}")
print(f"Q-values at this state: {q_table[discrete_state]}")


 State Conversion Example:
Continuous State: [-0.5  0. ]
Discrete State: (np.int64(15), np.int64(20))
Q-values at this state: [-0.29170937 -1.30504642 -1.77702759]


## Part 3: Q-Learning Algorithm Implementation

### 3.1 Q-Learning Hyperparameters

**Learning Rate (α):** Controls how much new information overrides old information
- Higher values = faster learning but less stable
- Lower values = slower but more stable learning

**Discount Factor (γ):** Determines importance of future rewards
- Close to 1 = long-term rewards important
- Close to 0 = immediate rewards important

**Epsilon (ε):** Controls exploration vs exploitation
- High epsilon = more exploration (random actions)
- Low epsilon = more exploitation (best known actions)

In [6]:
LEARNING_RATE = 0.1          
DISCOUNT = 0.95              
EPISODES = 100000

EPSILON = 1.0                
START_EPSILON_DECAY = 0.999      
END_EPSILON_DECAY = EPISODES // 2  
epsilon_decay_value = EPSILON / (END_EPSILON_DECAY - START_EPSILON_DECAY)

SHOW_EVERY = 2000           

print(" Q-Learning Hyperparameters:")
print("="*60)
print(f"Learning Rate (α): {LEARNING_RATE}")
print(f"Discount Factor (γ): {DISCOUNT}")
print(f"Episodes: {EPISODES:,}")
print(f"\nExploration Settings:")
print(f"Initial Epsilon: {EPSILON}")
print(f"Epsilon Decay Value: {epsilon_decay_value:.6f}")
print(f"Decay Period: Episodes {START_EPSILON_DECAY} to {END_EPSILON_DECAY}")

 Q-Learning Hyperparameters:
Learning Rate (α): 0.1
Discount Factor (γ): 0.95
Episodes: 100,000

Exploration Settings:
Initial Epsilon: 1.0
Epsilon Decay Value: 0.000020
Decay Period: Episodes 0.999 to 50000


### 3.2 The Q-Learning Update Formula

The core of Q-Learning is the update rule:

**Q_new(s,a) = Q_old(s,a) + α × [R + γ × max(Q(s',a')) - Q_old(s,a)]**

Where:
- **Q_old(s,a)**: Current Q-value for state s and action a
- **α**: Learning rate
- **R**: Immediate reward
- **γ**: Discount factor
- **max(Q(s',a'))**: Maximum Q-value for next state s'
- **Q_new(s,a)**: Updated Q-value

This formula updates our Q-values based on:
1. The immediate reward received
2. The estimated value of the best action in the next state
3. The difference between our prediction and reality (temporal difference)

### 3.3 Training Loop

Now we implement the complete Q-Learning training loop. This will:
1. Run 25,000 episodes
2. In each episode, take actions until goal is reached or max steps exceeded
3. Update Q-values using the Q-Learning formula
4. Gradually reduce exploration (epsilon decay)
5. Track rewards for analysis

In [ ]:
ep_rewards = []
aggr_ep_rewards = {'ep': [], 'avg': [], 'min': [], 'max': []}

print("Starting Q-Learning Training...")
print("="*60)
print(f"Training for {EPISODES:,} episodes")
print(f"This may take a few minutes...\n")

start_time = time.time()

for episode in range(EPISODES):
    episode_reward = 0

    state, info = env.reset()
    discrete_state = get_discrete_state(state)

    done = False

    while not done:
        if np.random.random() > EPSILON:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, env.action_space.n)

        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward

        new_discrete_state = get_discrete_state(new_state)

        if not done:
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        elif new_state[0] >= env.observation_space.high[0]:
            q_table[discrete_state + (action,)] = 0

        discrete_state = new_discrete_state

    if END_EPSILON_DECAY >= episode >= START_EPSILON_DECAY:
        EPSILON -= epsilon_decay_value

    ep_rewards.append(episode_reward)

    if episode % SHOW_EVERY == 0:
        avg_reward = sum(ep_rewards[-SHOW_EVERY:]) / len(ep_rewards[-SHOW_EVERY:])
        aggr_ep_rewards['ep'].append(episode)
        aggr_ep_rewards['avg'].append(avg_reward)
        aggr_ep_rewards['min'].append(min(ep_rewards[-SHOW_EVERY:]))
        aggr_ep_rewards['max'].append(max(ep_rewards[-SHOW_EVERY:]))

        print(f"Episode: {episode:6d} | "
              f"Avg: {avg_reward:7.2f} | "
              f"Min: {min(ep_rewards[-SHOW_EVERY:]):7.2f} | "
              f"Max: {max(ep_rewards[-SHOW_EVERY:]):7.2f} | "
              f"Epsilon: {EPSILON:.4f}")

env.close()

elapsed_time = time.time() - start_time
print(f"\n Training completed in {elapsed_time:.2f} seconds")
print(f"Final epsilon: {EPSILON:.4f}")
print(f"Final average reward: {aggr_ep_rewards['avg'][-1]:.2f}")

Starting Q-Learning Training...
Training for 100,000 episodes
This may take a few minutes...

Episode:      0 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 1.0000
Episode:   2000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.9600
Episode:   4000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.9200
Episode:   6000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.8800
Episode:   8000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.8400
Episode:  10000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.8000
Episode:  12000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.7600
Episode:  14000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.7200
Episode:  16000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.6800
Episode:  18000 | Avg: -200.00 | Min: -200.00 | Max: -200.00 | Epsilon: 0.6400
Episode:  20000 | Avg: -199.99 | Min: -200.00 | Max: -180.00 | Epsilon: 0.6000
Episode:  22000 | Avg: -199.86 | Min:

## Part 4: Results Analysis and Visualization

### 4.1 Reward Plot Over Time

This plot shows how the agent's performance improves over training. We can see:
- **Average reward** trending upward as the agent learns
- **Min/max bands** showing the range of performance
- Eventually stabilizing as the agent masters the task

In [ ]:
plt.figure(figsize=(14, 6))

plt.plot(aggr_ep_rewards['ep'], aggr_ep_rewards['avg'], label='Average Reward', linewidth=2)
plt.plot(aggr_ep_rewards['ep'], aggr_ep_rewards['min'], label='Min Reward', alpha=0.5, linestyle='--')
plt.plot(aggr_ep_rewards['ep'], aggr_ep_rewards['max'], label='Max Reward', alpha=0.5, linestyle='--')

plt.fill_between(aggr_ep_rewards['ep'],
                 aggr_ep_rewards['min'],
                 aggr_ep_rewards['max'],
                 alpha=0.2)

plt.xlabel('Episode', fontsize=12)
plt.ylabel('Reward', fontsize=12)
plt.title('Q-Learning Performance: Mountain Car Environment', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n Training Progress Analysis:")
print("="*60)
print(f"Initial average reward: {aggr_ep_rewards['avg'][0]:.2f}")
print(f"Final average reward: {aggr_ep_rewards['avg'][-1]:.2f}")
print(f"Improvement: {aggr_ep_rewards['avg'][-1] - aggr_ep_rewards['avg'][0]:.2f}")
print(f"\nBest performance: {max(aggr_ep_rewards['max']):.2f}")
print(f"Worst performance: {min(aggr_ep_rewards['min']):.2f}")

### 4.2 Detailed Performance Metrics

In [ ]:
recent_rewards = ep_rewards[-1000:] 

print(" Final Performance Metrics (Last 1000 Episodes):")
print("="*60)
print(f"Mean Reward: {np.mean(recent_rewards):.2f}")
print(f"Median Reward: {np.median(recent_rewards):.2f}")
print(f"Std Deviation: {np.std(recent_rewards):.2f}")
print(f"Min Reward: {np.min(recent_rewards):.2f}")
print(f"Max Reward: {np.max(recent_rewards):.2f}")

success_threshold = -200
successes = sum(1 for r in recent_rewards if r > success_threshold)
success_rate = (successes / len(recent_rewards)) * 100

print(f"\n Success Rate (reward > {success_threshold}): {success_rate:.1f}%")

## Part 5: Testing the Trained Agent

### 5.1 Demonstrate Learned Policy

Now let's test our trained agent by running it in the environment and observing its behavior. The agent should consistently reach the goal using the learned Q-values.

In [ ]:
print("Testing Trained Agent...")
print("="*60)

test_episodes = 10
test_rewards = []

for episode in range(test_episodes):
    state, info = env.reset()
    discrete_state = get_discrete_state(state)

    done = False
    episode_reward = 0
    steps = 0

    while not done:
        action = np.argmax(q_table[discrete_state])
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        episode_reward += reward
        steps += 1
        discrete_state = get_discrete_state(new_state)

    test_rewards.append(episode_reward)
    result = "SUCCESS" if new_state[0] >= 0.5 else "FAILED"
    print(f"Test Episode {episode+1}: Reward = {episode_reward:6.0f} | Steps = {steps:3d} | {result}")

print(f"\n Test Results:")
print(f"Average Reward: {np.mean(test_rewards):.2f}")
print(f"Success Rate: {sum(1 for r in test_rewards if r > -200) / len(test_rewards) * 100:.0f}%")

env.close()

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

env = gym.make('MountainCar-v0', render_mode='rgb_array')

frames = []
state, info = env.reset()
discrete_state = get_discrete_state(state)
done = False

while not done:
    frame = env.render()
    frames.append(frame)
    action = np.argmax(q_table[discrete_state])
    state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    discrete_state = get_discrete_state(state)

env.close()

fig = plt.figure(figsize=(6, 4))
plt.axis('off')
im = plt.imshow(frames[0])

def animate_func(i):
    im.set_array(frames[i])
    return [im]

anim = animation.FuncAnimation(fig, animate_func, frames=len(frames), interval=50)
HTML(anim.to_jshtml())


## Part 6: Analysis and Conclusions

### 6.1 Key Findings

**Learning Process:**
1. **Initial Phase (Episodes 0-5000)**: High exploration, random behavior, poor performance
2. **Learning Phase (Episodes 5000-15000)**: Agent discovers momentum strategy, rewards improve significantly
3. **Convergence Phase (Episodes 15000-25000)**: Performance stabilizes, consistent goal reaching

**Why Q-Learning Works:**
- The algorithm learns to value state-action pairs that lead to the goal
- Through temporal difference learning, it propagates rewards backward through action sequences
- The discretized Q-table allows efficient lookup and update of learned values

**Challenges Addressed:**
- **Continuous State Space**: Solved by discretization into 20×20 grid
- **Sparse Rewards**: Overcome through proper discount factor and sufficient training
- **Exploration-Exploitation**: Managed through epsilon decay strategy

### 6.2 Q-Learning Formula Explained

The Q-Learning update formula we used:

```
new_q = (1 - α) * current_q + α * (reward + γ * max_future_q)
```

**Component Breakdown:**
- **(1 - α) * current_q**: Retain some of the old Q-value (stability)
- **α * (reward + γ * max_future_q)**: Incorporate new information
  - **reward**: Immediate feedback from environment
  - **γ * max_future_q**: Discounted future rewards

This creates a weighted average between:
- What we previously believed (old Q-value)
- What we just learned (reward + future value)

**Why It Works:**
- Gradually adjusts beliefs based on experience
- Balances short-term rewards with long-term strategy
- Converges to optimal policy given enough training

### 6.3 Hyperparameter Impact

**Learning Rate (α = 0.1):**
- Moderate value allows steady learning without instability
- Too high: unstable, oscillating values
- Too low: very slow convergence

**Discount Factor (γ = 0.95):**
- High value emphasizes long-term rewards
- Critical for this problem since goal requires sequence of actions
- Lower values would lead to myopic behavior

**Epsilon Decay:**
- Linear decay from 1.0 to 0.0
- Ensures sufficient exploration early on
- Transitions to exploitation as agent gains knowledge

**Discretization (20×20):**
- Balance between granularity and computational efficiency
- Finer discretization (e.g., 40×40) could improve performance but requires more training
- Coarser discretization (e.g., 10×10) trains faster but less precise control

### 6.4 Real-World Applications

**Q-Learning Principles Apply To:**

1. **Robotics**: Robot navigation, manipulation tasks
2. **Game AI**: Chess, Go, video game agents
3. **Resource Management**: Traffic light optimization, power grid control
4. **Finance**: Trading strategies, portfolio optimization
5. **Healthcare**: Treatment planning, drug dosage optimization

**Advantages:**
- Model-free: doesn't need environment dynamics
- Off-policy: can learn from any experience
- Guaranteed convergence to optimal policy

**Limitations:**
- Requires discretization for continuous spaces
- Doesn't scale well to high-dimensional problems
- Needs many samples for convergence

**Solution for Limitations:**
Deep Q-Learning (DQN) uses neural networks instead of tables, enabling high-dimensional continuous state spaces.

## Summary

This assignment successfully implemented Q-Learning for the Mountain Car problem, demonstrating:

 **Complete Q-Learning Implementation**: From scratch using gymnasium
 **Proper State Discretization**: Converting continuous to discrete states
 **Epsilon-Greedy Exploration**: Balancing exploration and exploitation
 **Q-Table Updates**: Using temporal difference learning
 **Performance Analysis**: Comprehensive reward tracking and visualization
 **Successful Learning**: Agent learns to reach goal consistently

**Key Takeaway:**
Q-Learning enables agents to learn optimal behavior through trial and error, without requiring a model of the environment. The algorithm's simplicity and effectiveness make it a fundamental technique in reinforcement learning, though modern applications often use deep learning extensions like DQN for more complex problems.

---

**References:**
- sentdex Q-Learning Tutorial Series
- Gymnasium Documentation: https://gymnasium.farama.org/
- Sutton & Barto, "Reinforcement Learning: An Introduction"